In [1]:
# XGBoost - obican

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import pandas as pd

df_with_rawg = pd.read_json('./content/video_games_newer_all_data_with_rawg_final_for_train.json')


training, test = train_test_split(df_with_rawg, test_size = 0.25, random_state = 32)

features = training.drop(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis = 1).columns
print(features)

training_features, training_labels = training[features], training['NA_Sales']
test_features, test_labels = test[features], test['NA_Sales']

pipeline = Pipeline([('scaler', MinMaxScaler()), ('regressor', XGBRegressor())])
pipeline.fit(training_features, training_labels)

r2 = pipeline.score(test_features, test_labels)
print(f'Score {r2}')

Index(['Year_of_Release', 'User_Score', 'added', 'screenshots_count',
       'achievements_count', 'reddit_count', 'twitch_count', 'youtube_count',
       'ratings_count', 'suggestions_count', 'additions_count',
       'game_series_count', 'reviews_count', 'Adventure', 'Puzzle',
       'Educational', 'Family', 'Platformer', 'Racing', 'Casual', 'RPG',
       'Shooter', 'Arcade', 'Fighting', 'Card', 'Massively Multiplayer',
       'Misc_Genre', 'Strategy', 'Sports', 'Board Games', 'Simulation',
       'Action', 'Indie', 'Xbox 360', 'Wii', 'Nintendo DS', 'Xbox',
       'PlayStation 3', 'Nintendo 3DS', 'Game Boy Color', 'PlayStation 5',
       'iOS', 'PlayStation 2', 'Nintendo Switch', 'Wii U', 'Xbox Series S/X',
       'Game Boy Advance', 'Nintendo 64', 'PSP', 'PlayStation', 'PC',
       'GameCube', 'Android', 'Game Boy', 'PS Vita', 'NES', 'PlayStation 4',
       'Xbox One', 'Misc_Platform'],
      dtype='object')
Score 0.5339405114694518


In [7]:
import sklearn

sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [13]:
# XGBoost regressor - halvinggridsearch

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

data = pd.read_json('./content/video_games_newer_all_data_with_rawg_final_for_train.json')
training, test = train_test_split(data, test_size = 0.25, random_state = 32)

features = training.drop(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis = 1).columns

training_features, training_labels = training[features], training['NA_Sales']
test_features, test_labels = test[features], test['NA_Sales']

params = { 'regressor__max_depth': [3,6,10],
           'regressor__learning_rate': [0.01, 0.0125, 0.015],
           'regressor__n_estimators': [100, 500, 1000],
           'regressor__colsample_bytree': [0.1, 0.3, 0.7]
         }

xgb_pipeline = Pipeline([('scaler', StandardScaler()), ('regressor', XGBRegressor())])
clf = HalvingGridSearchCV(
    estimator=xgb_pipeline, 
    param_grid=params,               
    scoring='neg_mean_squared_error', 
    verbose=1
)
clf.fit(training_features, training_labels)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))
neg_mse = clf.score(test_features, test_labels)
print(f'Score {(-neg_mse)**(1/2.0)}')

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 35
max_resources_: 2851
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 81
n_resources: 35
Fitting 5 folds for each of 81 candidates, totalling 405 fits
----------
iter: 1
n_candidates: 27
n_resources: 105
Fitting 5 folds for each of 27 candidates, totalling 135 fits
----------
iter: 2
n_candidates: 9
n_resources: 315
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 3
n_candidates: 3
n_resources: 945
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 1
n_resources: 2835
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'regressor__colsample_bytree': 0.7, 'regressor__learning_rate': 0.01, 'regressor__max_depth': 3, 'regressor__n_estimators': 100}
Lowest RMSE:  1.375423229266373
Score 1.2338742039428987


In [ ]:
# SVM - halvinggridsearch

from sklearn import svm
from time import time
from pandas import DataFrame
from pathlib import Path
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

data = pd.read_json('./content/video_games_newer_all_data_with_rawg_final_for_train.json')
training, test = train_test_split(data, test_size = 0.25, random_state = 32)

features = training.drop(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis = 1).columns

training_features, training_labels = training[features], training['NA_Sales']
test_features, test_labels = test[features], test['NA_Sales']

params = { 'regressor__max_depth': [3,6,10],
           'regressor__learning_rate': [0.001, 0.005, 0.01],
           'regressor__n_estimators': [100, 500, 1000],
           'regressor__colsample_bytree': [0.1, 0.3, 0.7]
         }

svm_pipeline = Pipeline([('scaler', StandardScaler()), ('regressor', XGBRegressor())])
clf = HalvingGridSearchCV(
    estimator=xgb_pipeline, 
    param_grid=params,               
    scoring='neg_mean_squared_error', 
    verbose=1
)
clf.fit(training_features, training_labels)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))
neg_mse = clf.score(test_features, test_labels)
print(f'Score {(-neg_mse)**(1/2.0)}')